In [1]:
import torch
import os
import numpy as np
from datasets.crowd import Crowd
from models.vgg import vgg19
import argparse
import sys
sys.path.insert(0, '/home/pongpisit/CSRNet_keras/')
from SFANet_crowd_counting import seg_aspp2

In [2]:
model = seg_aspp2.Model()
device = torch.device('cuda')
model.to(device)
# checkpoint = torch.load(os.path.join(r'/home/pongpisit/CSRNet_keras/Bayesian-Crowd-Counting/seg_logs/0327-172121/', 'best_model.pth'))
# model.load_state_dict(checkpoint)
# del checkpoint

# model.load_state_dict(torch.load(os.path.join('./u_logs/0331-111426/', 'best_model.pth'), device))
# model.load_state_dict(torch.load(os.path.join('./seg_logs/0327-172121/', 'best_model.pth'), device))

Model(
  (vgg): VGG(
    (conv1_1): BaseConv(
      (activation): ReLU()
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): SynchronizedBatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv1_2): BaseConv(
      (activation): ReLU()
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): SynchronizedBatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv2_1): BaseConv(
      (activation): ReLU()
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): SynchronizedBatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv2_2): BaseConv(
      (activation): ReLU()
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): SynchronizedBatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (co

In [3]:
from thop import profile
input = torch.randn(1, 3, 224, 224).to(device)
macs, params = profile(model, inputs=(input, ))
print('params(M)', params/1e6)
print('MACs(G)', macs/1e9)

params(M) 9.747526
MACs(G) 18.141659136


In [4]:
import time

In [5]:
from tqdm import tqdm

In [6]:
res = [(320, 240),
(640, 480),
(1280, 960),
(1600, 1200)]

In [8]:
for w, h in res:
    input = torch.randn(50, 3, w, h).to(device)
    inference_time = 0
    model.eval()
    for i in tqdm(range(100)):
        start_time = time.time()
        model(input[i%50].unsqueeze_(0))
        inference_time += (time.time() - start_time)
    print(w, h)
    print(inference_time/100)
    print('======')

100%|██████████| 100/100 [00:01<00:00, 71.80it/s]


320 240
0.013841447830200195


100%|██████████| 100/100 [00:05<00:00, 17.99it/s]


640 480
0.051414775848388675


100%|██████████| 100/100 [00:21<00:00,  4.32it/s]


1280 960
0.2105325722694397


100%|██████████| 100/100 [00:33<00:00,  2.83it/s]

1600 1200
0.33718645572662354
